In [23]:
import pandas as pd
import numpy as np

In [24]:
df = pd.read_csv("InitialServices.csv")

In [25]:
df

Unnamed: 0  Rake Num Start Station Start Time End Station End Time  \
0             27       702           VND      05:40     KKDA DN    06:08   
1            108       701          MKPD      05:40        MKPR    06:00   
2              4       703           VND      05:40     PVGW UP    06:58   
3              2       709           IPE      05:45     PVGW UP    06:56   
4             78       711          SAKP      05:45     KKDA DN    06:59   
...          ...       ...           ...        ...         ...      ...   
1465        2072       719          SVVR      24:35        MUPR    24:41   
1466        2079       730          MUPR      24:37        SVVR    24:44   
1467        2074       719          MUPR      24:41    MUPR 4TH    25:01   
1468        2011       733          MKPR      24:43     MKPR UP    25:03   
1469        2081       730          SVVR      24:46    SVVR DN     24:46   

     Direction  Service Time  
0           DN            28  
1           DN            20  
2           UP            78  
3           UP            71  
4           DN            74  
...        ...           ...  
1465        UP             6  
1466        DN             7  
1467        UP            20  
1468        UP            20  
1469        UP             0  

[1470 rows x 8 columns]

In [26]:
added_services = [] #index of services added will get into this
CrewControl = ["KKDA DN", "KKDA UP", "PVGW UP", "PVGW DN"]


In [27]:
def hhmm2mins(hhmm):
    hrs = hhmm[:2]
    min = hhmm[-2:]
    mins = int(hrs)*60 + int(min)
    return mins

In [28]:
def timeDiff(x, y):

  return hhmm2mins(y) - hhmm2mins(x)

In [29]:
rakeNum = []
startStn = []
startTime = []
endStn = []
endTime = []
direction = []
serviceTime = []
stepBackRake = []
stepBackLocation = []

#Firstly adding services which operate between crew controls and doesnt require stepback (e.g.. KKDA TO PVGW)
for i in range(df.shape[0]):
  if df.iloc[i,0] not in added_services:
    if df.iloc[i,2] in CrewControl:
      if df.iloc[i,4] in CrewControl:
        added_services.append(df.iloc[i,0])
        rakeNum.append(df.iloc[i,1])
        startStn.append(df.iloc[i,2])
        startTime.append(df.iloc[i,3])
        endStn.append(df.iloc[i,4])
        endTime.append(df.iloc[i,5])
        direction.append(df.iloc[i,6])
        serviceTime.append(df.iloc[i,7])
        stepBackRake.append("No StepBack")
        stepBackLocation.append("No StepBack")


In [30]:
#Merging services which end and start in MKPR

for i in range(df.shape[0]):
  if df.iloc[i,0] not in added_services:
    if df.iloc[i,4] == "MKPR":
      for j in range(i, df.shape[0]):
        if df.iloc[j,2] == "MKPR" and df.iloc[i,1] == df.iloc[j,1] and timeDiff(df.iloc[i,5], df.iloc[j,3]) < 30:
          #print(df.iloc[i,1], df.iloc[i,2], df.iloc[i,3], df.iloc[i,4], df.iloc[i,5] , "with", df.iloc[j,1], df.iloc[j,2], df.iloc[j,3], df.iloc[j,4], df.iloc[j,5])
          added_services.append(df.iloc[i,0])
          added_services.append(df.iloc[j,0])
          rakeNum.append(df.iloc[i,1])
          startStn.append(df.iloc[i,2])
          startTime.append(df.iloc[i,3])
          endStn.append(df.iloc[j,4])
          endTime.append(df.iloc[j,5])
          direction.append(df.iloc[i,6])
          serviceTime.append(timeDiff(df.iloc[i,3], df.iloc[j,5]))
          stepBackRake.append("No StepBack")
          stepBackLocation.append("No StepBack")

In [31]:
# merging service from MUPR to SSVR and SVVR to MUPR
for i in range(df.shape[0]):
  if df.iloc[i,0] not in added_services:
    if df.iloc[i,4] == "SVVR" :
      flag = False
      for j in range(i, df.shape[0]):
        if df.iloc[j,2] == "SVVR" and timeDiff(df.iloc[i,5], df.iloc[j,3]) > 0 and df.iloc[j,0] not in added_services and df.iloc[i,1] == df.iloc[j,1]:
          added_services.append(df.iloc[i,0])
          added_services.append(df.iloc[j,0])
          rakeNum.append(df.iloc[i,1])
          startStn.append(df.iloc[i,2])
          startTime.append(df.iloc[i,3])
          endStn.append(df.iloc[j,4])
          endTime.append(df.iloc[j,5])
          direction.append(df.iloc[i,6])
          serviceTime.append(timeDiff(df.iloc[i,3], df.iloc[j,5]))
          stepBackRake.append("No StepBack")
          stepBackLocation.append("No StepBack")
          #print(df.iloc[i,1], df.iloc[i,2], df.iloc[i,3], df.iloc[i,4], df.iloc[i,5] , "with", df.iloc[j,1], df.iloc[j,2], df.iloc[j,3], df.iloc[j,4], df.iloc[j,5])
          flag = True
          break

In [32]:
#StepBack for MUPR is added

for i in range(df.shape[0]):
  if df.iloc[i,0] not in added_services:
    if df.iloc[i,2] != "SVVR" and df.iloc[i,4] == "MUPR" :
      flag = False
      for j in range(i, df.shape[0]):
        if df.iloc[j,2] == "MUPR" and df.iloc[j,4] != "SVVR" and timeDiff(df.iloc[i,5], df.iloc[j,3]) > 0 and df.iloc[j,0] not in added_services:
          added_services.append(df.iloc[i,0])
          added_services.append(df.iloc[j,0])
          rakeNum.append(df.iloc[i,1])
          startStn.append(df.iloc[i,2])
          startTime.append(df.iloc[i,3])
          endStn.append(df.iloc[j,4])
          endTime.append(df.iloc[j,5])
          direction.append(df.iloc[i,6])
          serviceTime.append(timeDiff(df.iloc[i,3], df.iloc[j,5]))
          stepBackRake.append(df.iloc[j,1])
          stepBackLocation.append(df.iloc[i,4])
          #print(df.iloc[i,1], df.iloc[i,2], df.iloc[i,3], df.iloc[i,4], df.iloc[i,5] , "with", df.iloc[j,1], df.iloc[j,2], df.iloc[j,3], df.iloc[j,4], df.iloc[j,5])
          flag = True
          break

      if not flag: print("No StepBack Found", df.iloc[i,0],df.iloc[i,1], df.iloc[i,2], df.iloc[i,3], df.iloc[i,4], df.iloc[i,5])


No StepBack Found 2055 726 KKDA DN 23:37 MUPR 23:52
No StepBack Found 2062 724 KKDA DN 23:52 MUPR 24:07
No StepBack Found 2070 719 KKDA DN 24:07 MUPR 24:22
No StepBack Found 2078 730 KKDA DN 24:22 MUPR 24:37


In [33]:
#Not adding stepback for services which are ending in MKPR

for i in range(df.shape[0]):
  if df.iloc[i,0] not in added_services:
    if df.iloc[i,4] == "MKPR":
      added_services.append(df.iloc[i,0])
      rakeNum.append(df.iloc[i,1])
      startStn.append(df.iloc[i,2])
      startTime.append(df.iloc[i,3])
      endStn.append(df.iloc[i,4])
      endTime.append(df.iloc[i,5])
      direction.append(df.iloc[i,6])
      serviceTime.append(df.iloc[i,7])
      stepBackRake.append("No StepBack")
      stepBackLocation.append("No StepBack")


In [34]:
#Which are not starting or ending in MUPR

for i in range(df.shape[0]):
  if df.iloc[i,0] not in added_services:
    if df.iloc[i,4] != "MUPR" and df.iloc[i,2] != "MUPR":
      #print(df.iloc[i,0],df.iloc[i,1], df.iloc[i,2], df.iloc[i,3], df.iloc[i,4], df.iloc[i,5])
      added_services.append(df.iloc[i,0])
      rakeNum.append(df.iloc[i,1])
      startStn.append(df.iloc[i,2])
      startTime.append(df.iloc[i,3])
      endStn.append(df.iloc[i,4])
      endTime.append(df.iloc[i,5])
      direction.append(df.iloc[i,6])
      serviceTime.append(df.iloc[i,7])
      stepBackRake.append("No StepBack")
      stepBackLocation.append("No StepBack")

In [35]:
len(added_services)

1462

In [36]:
len(rakeNum)

926

In [37]:
#Services which are initiating and ending in MUPR (like sign ON and signOFF)

for i in range(df.shape[0]):
  if df.iloc[i,0] not in added_services:
    added_services.append(df.iloc[i,0])
    rakeNum.append(df.iloc[i,1])
    startStn.append(df.iloc[i,2])
    startTime.append(df.iloc[i,3])
    endStn.append(df.iloc[i,4])
    endTime.append(df.iloc[i,5])
    direction.append(df.iloc[i,6])
    serviceTime.append(df.iloc[i,7])
    stepBackRake.append("No StepBack")
    stepBackLocation.append("No StepBack")

In [38]:
len(rakeNum)

934

In [39]:
services = pd.DataFrame(list(zip(rakeNum,startStn,startTime,endStn,endTime,direction,serviceTime, stepBackRake, stepBackLocation)),
               columns =['Rake Num','Start Station','Start Time','End Station','End Time','Direction','service time', "Step Back Rake", "Step Back Location"])

In [40]:
services = services.sort_values(by=['Start Time'])
services

Rake Num Start Station Start Time End Station End Time Direction  \
336       701          MKPD      05:40     PVGW DN    06:15        DN   
872       702           VND      05:40     KKDA DN    06:08        DN   
873       703           VND      05:40     PVGW UP    06:58        UP   
874       709           IPE      05:45     PVGW UP    06:56        UP   
875       711          SAKP      05:45     KKDA DN    06:59        DN   
..        ...           ...        ...         ...      ...       ...   
525       716       PVGW UP      24:13       MKPD     24:48        UP   
691       719          MUPR      24:22        MUPR    24:41        DN   
933       730       KKDA DN      24:22        MUPR    24:37        DN   
526       733       PVGW UP      24:28     MKPR UP    25:03        UP   
692       730          MUPR      24:37    SVVR DN     24:46        DN   

     service time Step Back Rake Step Back Location  
336            35    No StepBack        No StepBack  
872            28    No StepBack        No StepBack  
873            78    No StepBack        No StepBack  
874            71    No StepBack        No StepBack  
875            74    No StepBack        No StepBack  
..            ...            ...                ...  
525            35    No StepBack        No StepBack  
691            19    No StepBack        No StepBack  
933            15    No StepBack        No StepBack  
526            35    No StepBack        No StepBack  
692             9    No StepBack        No StepBack  

[934 rows x 9 columns]

In [41]:
services.iloc[0,0]

701

In [42]:
category = []


for i in range(services.shape[0]):
  # if services.iloc[i,0] not in used_rake:
  #   category.append("inducting")
  #   used_rake.append(services.iloc[i,0])
  # else:
  #   flag = False
  #   for j in range(i,services.shape[0]):
  #     if services.iloc[j,0] == services.iloc[i,0]:
  #       category.append("continuing")
  #       flag = True
  #       break
  #   if not flag:
  #     category.append("stabling")

  if services.iloc[i,1] not in CrewControl:
    #print( services.iloc[i,0],services.iloc[i,1], services.iloc[i,2], services.iloc[i,3], services.iloc[i,4], services.iloc[i,5])
    category.append("inducting")
  elif services.iloc[i,3] not in CrewControl:
   category.append("stabling")
  else: category.append("continuing")

print(len(category))

934


In [43]:
services['Category'] = category
services = services[['Rake Num','Start Station','Start Time','End Station','End Time','Direction','service time', "Category" ,"Step Back Rake", "Step Back Location"]]

In [44]:
services.to_csv('stepbackServices.csv')